# 2018 FIFA World Cup

<img src='./thumbnails/world_cup.png' alt="Panel Dashboard" align="right">

The FIFA World Cup is the premier international football tournament, held every four years and featuring teams from around the globe. It is a celebration of the sport, bringing together fans and players in a thrilling competition for the prestigious title. Each World Cup tournament offers a wealth of data on matches, players, and events, providing a rich resource for analysis and visualization.

In this notebook, we focus on the 2018 FIFA World Cup, hosted by Russia. Using `hvPlot` and `Panel`, we will create dynamic and interactive visualizations to explore the extensive dataset from this tournament. These tools enable us to investigate the statistics and uncover insights about player performances and more. 

The dataset used for this analysis is sourced from [Pappalardo, Luca; Massucco, Emanuele (2019)](https://doi.org/10.6084/m9.figshare.c.4415000.v5) Soccer match event dataset, figshare collection.

## Load the data

Here we will load the `players` and `World Cup events` dataset from the figshare collection to enable use create plots and visualizations focused only on the 2018 World Cup.

In [ ]:
import pandas as pd
import hvplot.pandas # noqa
import holoviews as hv
from holoviews import opts
import panel as pn

pn.extension()

In [ ]:
players_df = pd.read_json('data/players.json', encoding='unicode-escape')
events_df = pd.read_json('data/events/events_World_Cup.json')

In [ ]:
events_df.head()

In [ ]:
players_df.tail()

## Event distribution

We can take a look at the unique events that take place in a typical football game and plot the frequency of those events using a hvPlot bar chart:

In [ ]:
pitch_events = list(events_df['eventName'].unique())
pitch_events

In [ ]:
event_type_count = events_df['eventName'].value_counts()
event_type_distribution = event_type_count.hvplot.bar(title='Distribution of Event Types', height=400,
                                                      width=800, rot=45)
event_type_distribution

It is unsurprising that passes are the most common event in a football match, however we would also like to see the areas of the football pitch most these events occur.

First, we will use the `holoviews` library to draw an outline of a football pitch:

In [ ]:
opts.defaults(opts.Path(color = 'black'),
            opts.Points(color='black', size=5))

In [ ]:
def plot_pitch():
    pitch_elements = [
    hv.Path([[(0, 0), (0, 100), (100, 100), (100, 0), (0, 0)]]), # pitch outline
    hv.Path([[(50, 0), (50, 100)]]), # center line
    hv.Path([[(16, 19), (16, 81), (0, 81), (0, 19), (16, 19)]]), # left penalty area
    hv.Path([[(84, 19), (84, 81), (100, 81), (100, 19), (84, 19)]]), # right penalty area
    hv.Path([[(0, 37), (6, 37), (6, 63), (0, 63), (0, 37)]]), # left six yard box
    hv.Path([[(100, 37), (94, 37), (94, 63), (100, 63), (100, 37)]]), # right six yard box
    hv.Ellipse(50, 50, 16.3), # center circle
    hv.Points([(50, 50)]), # center spot
    hv.Points([(11, 50)]), # left penalty spot
    hv.Points([(89, 50)]), # right penalty spot
    hv.Arrow(20, 5, '', '>', ), # attack arrow
    hv.Text(7, 7, 'Attack', 12)] # attack text

    pitch = hv.Overlay(pitch_elements).opts(frame_width=600, frame_height=300, show_frame=False,
                                            xlim=(-5, 105), ylim=(-5, 105), xaxis=None, yaxis=None)

    return pitch

In [ ]:
pitch = plot_pitch()
pitch

Then, we can generate a heatmap to see where these events occur the most on the pitch:

In [ ]:
def plot_event_heatmap(events_df, event_type, cmap='Greens'):
    """
    Plots a heatmap of the specified event type on a football pitch.
    
    Parameters:
    events_df (pd.DataFrame): The dataframe containing event data with the following columns:
        - eventId: The identifier of the event's type.
        - eventName: The name of the event's type.
        - subEventId: The identifier of the subevent's type.
        - subEventName: The name of the subevent's type.
        - tags: A list of event tags describing additional information about the event.
        - eventSec: The time when the event occurs (in seconds since the beginning of the current half).
        - id: A unique identifier of the event.
        - matchId: The identifier of the match the event refers to.
        - matchPeriod: The period of the match (1H, 2H, E1, E2, P).
        - playerId: The identifier of the player who generated the event.
        - positions: The origin and destination positions associated with the event.
        - teamId: The identifier of the player's team.
    event_type (str): The type of event to plot (e.g., 'Pass', 'Duel', 'Shot').
    cmap (str): The color map to use for the heatmap. Default is 'Greens'.
    
    Returns:
    hvPlot object: A heatmap plot of the specified event type overlaid on a football pitch.
    """
    event_type = event_type.lower()
    event = events_df[events_df['eventName'].str.lower() == event_type]
    positions = [(pos[0]['x'], pos[0]['y']) for pos in event['positions'] if len(pos) > 0]
    event_df = pd.DataFrame(positions, columns=['x', 'y'])
    pitch = plot_pitch() 
    title = f"{event_type.capitalize()}s Heatmap" if event_type != 'pass' else "Passes Heatmap"
    
    event_heatmap = event_df.hvplot.hexbin(x='x', y='y', cmap=cmap, min_count=1, title=title)
    
    event_heatmap_plot = (event_heatmap * pitch).opts(
        frame_width=600, frame_height=300, xlim=(-5, 105), ylim=(-5, 105),
        show_frame=False, xaxis=None, yaxis=None)

    return event_heatmap_plot

For example, let use see the heatmap of the passes in a typical game:

In [ ]:
passes_map = plot_event_heatmap(events_df, 'pass')
passes_map

We can replace "pass" with another event type to see the heatmap for that event. However, Panel makes it easy to create widgets that we can use to select the different event types and immediately see the heatmap of that event.

First, we create a `Select` widget and use `pn.bind` to link the widget with the `event_heatmap` function. Then we can display it as a column using `pn.Column`:

In [ ]:
event_type_selector = pn.widgets.Select(name='Event Type', options=pitch_events)
event_heatmap = pn.bind(plot_event_heatmap, events_df=events_df, event_type=event_type_selector)

pn.Column(event_type_selector, event_heatmap)

If you have a live python process running, you can use the Selector widget to alternate between the different event types and see their heatmap on the football pitch.

## Player events

Using the `playerId` from the events dataframe, we can plot the top `n` players in any event category. First, we create a function to find the top players for any event type:

In [ ]:
def find_top_players(events_df, players_df, event_type, top_n=10):
    """
    Finds the top players for a given event type.

    Parameters:
    events_df (pd.DataFrame): The dataframe containing event data.
    players_df (pd.DataFrame): The dataframe containing player data.
    event_type (str): The type of event to filter by.
    top_n (int): The number of top players to return.

    Returns:
    pd.DataFrame: A dataframe containing the top players for the given event type.
    """
    event_type = event_type.lower()
    event = events_df[events_df['eventName'].str.lower() == event_type]
    event_counts = event.groupby('playerId').size().reset_index(name=f'{event_type} count')
    
    top_players = event_counts.sort_values(by=f'{event_type} count', ascending=False).head(top_n)
    top_players = top_players.merge(players_df, left_on='playerId', right_on='wyId')
    top_players.set_index('playerId', inplace=True)
    
    return top_players[['shortName', f'{event_type} count']]

For example, we can check the top 10 players with the highest passes in the World Cup:

In [ ]:
pass_maestros = find_top_players(events_df, players_df, 'pass')
pass_maestros

We can then create a bar chart to visualize these players:

In [ ]:
def plot_top_players(events_df, players_df, event_type, top_n=10):
    """
    Plots a bar chart of the top players for a given event type.

    Parameters:
    events_df (pd.DataFrame): The dataframe containing event data.
    players_df (pd.DataFrame): The dataframe containing player data.
    event_type (str): The type of event to filter by.
    top_n (int): The number of top players to return.

    Returns:
    hvPlot: A bar chart of the top players for the given event type.
    """
    top_players = find_top_players(events_df, players_df, event_type, top_n)
    event_type = event_type.lower()
    if event_type == 'pass':
        title = f'Top {top_n} Players for {event_type.capitalize()}es'
    else:
        title = f'Top {top_n} Players for {event_type.capitalize()}s'
    
    bar_plot = top_players.hvplot.bar(title=title, x='shortName', y=f'{event_type} count',
                                      xlabel='', ylabel=f'Number of {event_type}', height=300, width=600, rot=45)
    
    return bar_plot

In [ ]:
pass_maestros_plot = plot_top_players(events_df, players_df, 'pass')
pass_maestros_plot

We can do also plot the individual player heatmaps for any type of event on the football pitch. First, we create a function that maps the player name to their unique ID, then create another function that plots the player heatmap using the resulting player ID:

In [ ]:
def get_player_id(player_name):
    player_name_to_id = dict(zip(players_df['shortName'], players_df['wyId']))
    return player_name_to_id.get(player_name)

In [ ]:
def plot_player_event_heatmap(events_df, players_df, player_name):
    """
    Plots a heatmap of events performed by a specific player on a football pitch.
    
    Parameters:
    events_df (pd.DataFrame): The dataframe containing event data.
    players_df (pd.DataFrame): The dataframe containing player data.
    player_name (str): The name of the player to plot events for.
    
    Returns:
    hvPlot object: A heatmap plot of the player's events overlaid on a football pitch.
    """
    if not player_name:
        return pn.pane.Markdown("Select a player to see the heatmap.", height=200)
    
    player_id = get_player_id(player_name)
    if player_id is None:
        return pn.pane.Markdown("Please select a valid player.", height=200)
    
    player_events = events_df[events_df['playerId'] == player_id]
    
    if player_events.empty:
        return pn.pane.Markdown("No events found for the selected player.", height=200)
    
    positions = [(pos[0]['x'], pos[0]['y'], event) 
                 for pos, event in zip(player_events['positions'], player_events['eventName']) 
                 if len(pos) > 0]
    event_df = pd.DataFrame(positions, columns=['x', 'y', 'eventName'])
    pitch = plot_pitch()
    
    event_heatmap = event_df.hvplot.points(x='x', y='y', c='eventName', cmap='Category20',
                                           title=f'{player_name} Event Heatmap')
    
    player_heatmap_plot = (event_heatmap * pitch).opts(frame_width=600, frame_height=300,
                                                       xlim=(-5, 105), ylim=(-5, 105),
                                                       show_frame=False, xaxis=None, yaxis=None)
    
    return player_heatmap_plot

In [ ]:
isco_map = plot_player_event_heatmap(events_df, players_df, 'Isco')
isco_map

Using the Panel `AutocompleteInput` widget, we can then devise a way to search for players using their names and immediately seeing their event heatmap on the football pitch:

In [ ]:
player_name_selector = pn.widgets.AutocompleteInput(name='Player Name', options=list(players_df['shortName']),
                                                    placeholder='Type player name...', case_sensitive=False,
                                                   search_strategy='includes')

player_heatmap = pn.bind(plot_player_event_heatmap, events_df=events_df,
                         players_df=players_df, player_name=player_name_selector)

pn.Column(player_name_selector, player_heatmap, sizing_mode='stretch_both')

Another insight we can glean is the "player passing network", which is a way of calculating the average position of a player on the pitch and then plotting the direction where their passes get to:

In [ ]:
def plot_player_passing_network(events_df, players_df, player_name):
    """
    Plots the passing network for a specific player on a football pitch.
    
    Parameters:
    events_df (pd.DataFrame): The dataframe containing event data.
    players_df (pd.DataFrame): The dataframe containing player data.
    player_name (str): The name of the player to plot the passing network for.
    
    Returns:
    hvPlot object: A plot of the player's passing network overlaid on a football pitch.
    """
    player_id = get_player_id(player_name)
    
    if player_id is None:
        return pn.pane.Markdown("Please select a valid player.", height=200)
    
    player_events = events_df[events_df['playerId'] == player_id]
    
    if player_events.empty:
        return pn.pane.Markdown("No events found for the selected player.", height=200)
    
    passes = player_events[player_events['eventName'] == 'Pass']
    pass_positions = [(pos[0]['x'], pos[0]['y'], pos[1]['x'], pos[1]['y']) for pos in passes['positions'] if len(pos) > 1]
    pass_df = pd.DataFrame(pass_positions, columns=['start_x', 'start_y', 'end_x', 'end_y'])
    
    avg_x = player_events['positions'].apply(lambda x: x[0]['x']).mean()
    avg_y = player_events['positions'].apply(lambda x: x[0]['y']).mean()
    
    avg_position = hv.Points([(avg_x, avg_y)]).opts(size=10, marker='x', title=f'{player_name} Passing Network')
    
    total_passes = len(pass_df)
    
    total_passes_text = hv.Text(75, 103, f'Total number of passes: {total_passes}', halign='center', fontsize=10)
    
    pass_lines = [hv.Segments([(avg_x, avg_y, row['end_x'], row['end_y'])]) for index, row in pass_df.iterrows()]
    pass_lines_overlay = hv.Overlay(pass_lines)
    
    pitch = plot_pitch()
    
    network_plot = (avg_position * pass_lines_overlay * total_passes_text * pitch).opts(
        frame_width=600, frame_height=300, xlim=(-5, 105), ylim=(-5, 105),
        show_frame=False, xaxis=None, yaxis=None)
    
    return network_plot
    

Then we use the previously defined `player_name_selector` widget to bind it to the `plot_player_passing_network` in other to make it easier to search for different players and view their passing network:

In [ ]:
player_network = pn.bind(plot_player_passing_network, events_df=events_df,
                                     players_df=players_df, player_name=player_name_selector)

pn.Column(player_name_selector, player_network, sizing_mode='stretch_both')

## Dashboard

We can now combine all the different plots into one layout using `pn.Column`, but first we will create an `IntSlider` widget for the top n bar charts and then bind the widget to the `plot_top_players` function:

In [ ]:
top_n_selector = pn.widgets.IntSlider(name='Top', start=1, end=20, value=10)

bar_chart = pn.bind(plot_top_players, events_df=events_df, players_df=players_df,
                                event_type=event_type_selector, top_n=top_n_selector)

pn.Column(pn.Row(top_n_selector, event_type_selector), bar_chart)

Finally, we can now arrange all the plots into a neat layout with the widgets at the top:

In [ ]:
layout = pn.Column(
    pn.Row(event_type_selector, top_n_selector, player_name_selector, sizing_mode='stretch_width'),
    bar_chart,
    event_heatmap,
    player_heatmap,
    player_network,
    sizing_mode='stretch_both')

layout

## Servable dashboard

Now that we have a fully interactive dashboard, we can now deploy it in a template to give it a more polished look:

In [ ]:
logo = '<img src="https://panel.holoviz.org/_static/logo_stacked.png" width=180 height=150>'

text = ''' **Use the selector widget to select the different type of events on the pitch and see the areas where they occur the most.
            Use the slider widget to select the number of players to display.
            search with name of player to see their event map.**'''

template = pn.template.BootstrapTemplate(
    header_background='#18BB12',
    title='Interactive football dashboard',
    sidebar=[logo, text],
    main=[pn.panel(layout, sizing_mode='scale_both')]
)
template.servable();

If you have a live python process, running the cell above will open a standalone dashboard in a new browser tab where you can select and explore the data to your heart’s content, and share it with anyone else interested in this topic.

You can also display the dashboard alone using `panel serve --rest-session-info --session-history -1 world_cup.ipynb --show`